# FP-Growth

频繁项挖掘

数据集：[Groceries dataset](https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset)

In [1]:
import collections
import pickle

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.fpm import FPGrowth

import utils

In [2]:
CSV_PATH = './data'
CSV_FILE = 'Groceries_dataset.csv'
PLK_FILE = 'item_list.pkl'

## 1. 数据预处理

In [3]:
# 创建 SparkSession
spark = SparkSession.builder \
    .appName("App") \
    .getOrCreate()

# 将日志级别设为 WARN
spark.sparkContext.setLogLevel("ERROR")

# 从 CSV 中读取数据，存成 Pandas DataFrame
abs_path = utils.gen_abspath(CSV_PATH, CSV_FILE)
df = utils.read_csv(abs_path)

# 将 Pandas DataFrame 加载到 Spark
spark_df = spark.createDataFrame(df)
spark_df.show(10)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/06 22:04:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------------+----------+----------------+
|Member_number|      Date| itemDescription|
+-------------+----------+----------------+
|         1808|21-07-2015|  tropical fruit|
|         2552|05-01-2015|      whole milk|
|         2300|19-09-2015|       pip fruit|
|         1187|12-12-2015|other vegetables|
|         3037|01-02-2015|      whole milk|
|         4941|14-02-2015|      rolls/buns|
|         4501|08-05-2015|other vegetables|
|         3803|23-12-2015|      pot plants|
|         2762|20-03-2015|      whole milk|
|         4119|12-02-2015|  tropical fruit|
+-------------+----------+----------------+
only showing top 10 rows



设计一个转换类 `Convert`，将 `itemDescription` 的字段值编码成整数，这样做的好处是可以节省内存。

`Convert` 提供两个方法：
- 一个方法将字段值映射成整数
- 另一个方法将整数映射回字段值

In [4]:
class Convert:

    def __init__(self, lst: list, file_path='./item_list.pkl'):
        self.lst = lst
        self.file_path = file_path

        # 对 lst 进行编码
        self.item_list = None
        self.item_dict = None
        self.parse()

    @staticmethod
    def to_dict(item_list):
        item_dict = dict()
        for i, item in enumerate(item_list):
            item_dict[item] = i
        return item_dict

    def parse(self):
        """对列表元素编码"""

        # 统计字符串频率
        frequency = collections.Counter(self.lst)

        # 按字符串频率，倒序排列
        sorted_items = sorted(frequency.items(),
                              key=lambda e: e[1],
                              reverse=True)

        self.item_list = [e[0] for e in sorted_items]
        self.item_dict = self.to_dict(self.item_list)

    def reset(self):
        self.item_list = self.read_item_list()
        self.item_dict = self.to_dict(self.item_list)

    def __len__(self):
        return len(self.item_list)

    def __getitem__(self, index):
        return self.item_list[index]

    def encoder(self, item):
        return self.item_dict.get(item)

    def save_item_list(self):
        with open(self.file_path, 'wb') as f:
            pickle.dump(self.item_list, f)

    def read_item_list(self):
        with open(self.file_path, 'rb') as f:
            item_list = pickle.load(f)
        return item_list

简单应用一下 `Convert`，首先将它实例化 `cv = Convert(items)`
- `cv[i]`：获取编码为 `i` 的 item 的名称
- `cv.encoder([ITEM_NAME])`：获取名称为 `[ITEM_NAME]` 的 item 的编码

In [5]:
# 获取 Spark DataFrame 中的 item Description 字段
items = [row.itemDescription for row in spark_df.collect()]
pkl_path = utils.gen_abspath(CSV_PATH, PLK_FILE)
cv = Convert(items, file_path=pkl_path)

# 将编码的映射逻辑存成 pkl 文件
cv.save_item_list()

# 读 pkl 文件，看保存的 item_list 是否与当前相同
print(f'cv.read_item_list() == cv.item_list: {cv.read_item_list() == cv.item_list}')

# 用 pkl 文件的 item_list 重置 cv
cv = Convert([], file_path=pkl_path)
cv.reset()

# 获取编码为 2 的 item 
print(f'cv[0]: {cv[0]}')

# 获取 item  为 pot plants 的编码
print(f'cv.encoder("pot plants"): {cv.encoder("pot plants")}')

# 计算无重复 item  的个数
print(f'len(cv): {len(cv)}')

cv.read_item_list() == cv.item_list: True
cv[0]: whole milk
cv.encoder("pot plants"): 72
len(cv): 167


将 `cv.encoder` 注册为 Spark UDF，然后对 Spark 的每一行使用该 UDF，将 item 的名称转换为编码。产出一个新列 `itemCode` 用来存放 item 的编码。

In [6]:
# 将函数注册为 Spark UDF，返回值类型设为 Integer
encoder_udf = F.udf(cv.encoder, IntegerType())

# 在 DataFrame 上应用此函数
spark_df = spark_df.withColumn("itemCode", encoder_udf(F.col("itemDescription")))

spark_df.show(10)

+-------------+----------+----------------+--------+
|Member_number|      Date| itemDescription|itemCode|
+-------------+----------+----------------+--------+
|         1808|21-07-2015|  tropical fruit|       6|
|         2552|05-01-2015|      whole milk|       0|
|         2300|19-09-2015|       pip fruit|      11|
|         1187|12-12-2015|other vegetables|       1|
|         3037|01-02-2015|      whole milk|       0|
|         4941|14-02-2015|      rolls/buns|       2|
|         4501|08-05-2015|other vegetables|       1|
|         3803|23-12-2015|      pot plants|      72|
|         2762|20-03-2015|      whole milk|       0|
|         4119|12-02-2015|  tropical fruit|       6|
+-------------+----------+----------------+--------+
only showing top 10 rows



In [7]:
# 把具有相同 Member_number 的所有 itemCode 聚成一个无重复列表
items_df = spark_df.groupBy("Member_number").agg(F.collect_set("itemCode").alias("itemCodes"))

# 显示结果
items_df.show(10)

+-------------+--------------------+
|Member_number|           itemCodes|
+-------------+--------------------+
|         1000|[0, 48, 13, 67, 5...|
|         1001|[0, 15, 2, 17, 24...|
|         1002|[0, 27, 1, 6, 42,...|
|         1003|[45, 121, 5, 2, 6...|
|         1004|[0, 1, 31, 2, 89,...|
|         1005|         [15, 2, 25]|
|         1006|[0, 12, 31, 64, 2...|
|         1008|[96, 34, 5, 20, 1...|
|         1009|[60, 16, 6, 123, ...|
|         1010|[2, 17, 125, 36, ...|
+-------------+--------------------+
only showing top 10 rows



## 2. 频繁项挖掘

In [8]:
def freq2support(min_freq, seq_cnt):
    """由频数计算支持度"""
    support = min_freq / seq_cnt
    return support

In [9]:
seq_cnt = items_df.count()
distinct_seq_cnt = len(set(['_'.join(map(str, sorted(s))) for s in [row.itemCodes for row in items_df.collect()]]))

print(f'序列的数量：{seq_cnt}')
print(f'无重复序列的数量：{distinct_seq_cnt}')

序列的数量：3898
无重复序列的数量：3873


### 2.1 频繁项集

In [10]:
min_freq = 50
min_support = freq2support(min_freq=min_freq, seq_cnt=seq_cnt)
print(f'min_freq: {min_freq}')
print(f'min_support: {min_support:.4f}')

fpGrowth = FPGrowth(itemsCol="itemCodes", minSupport=min_support, minConfidence=0.5)
model = fpGrowth.fit(items_df)

# Display frequent item sets.
model.freqItemsets.show(10)

min_freq: 50
min_support: 0.0128


+----------+----+
|     items|freq|
+----------+----+
|      [92]|  71|
|     [102]|  60|
|      [83]|  85|
|      [24]| 471|
|   [24, 8]| 125|
|[24, 8, 2]|  59|
|[24, 8, 4]|  58|
|[24, 8, 1]|  58|
|[24, 8, 3]|  52|
|[24, 8, 0]|  74|
+----------+----+
only showing top 10 rows



### 2.2 关联规则

In [11]:
# Display generated association rules.
model.associationRules.show(10)

# transform examines the input item s against all the association rules and summarize the
# consequents as prediction
model.transform(items_df).show(10)

+----------+----------+------------------+------------------+--------------------+
|antecedent|consequent|        confidence|              lift|             support|
+----------+----------+------------------+------------------+--------------------+
|   [29, 6]|       [0]|0.5842696629213483|1.2751865319526403|0.013340174448435094|
|  [14, 13]|       [0]|0.5689655172413793|1.2417847627138279| 0.01693175987685993|
|   [22, 3]|       [0]|0.5232558139534884|1.1420219276543662|0.023088763468445357|
| [5, 4, 1]|       [0]|0.5909090909090909|1.2896772879975569|0.020010261672652643|
|  [16, 14]|       [0]|0.5631067961165048|1.2289979234390458|0.014879425346331451|
|      [18]|       [0]|0.5132075471698113|1.1200912759618837| 0.06977937403796819|
|  [24, 12]|       [0]|0.6086956521739131|  1.32849700569648|0.014366341713699333|
|   [12, 3]|       [0]|0.5674603174603174|1.2384996178389234| 0.03668547973319651|
|  [20, 14]|       [0]|0.6391752577319587|1.3950196834485862| 0.01590559261159569|
|   

<div style="border-radius:10px;border:#254E58 solid;padding: 15px;background-color:white;font-size:110%;text-align:left">

<h3 align="left"><font color=#254E58>📝 关联规则挖掘：</font></h3> 

关联规则挖掘是一种运用模式识别来识别和量化不同但相关 item 之间关系的处理过程。

一个简单的关联规则用例：

鸡蛋和面包经常一起购买。有了这个发现，你可以通过以下方式增加销售额：

- 将鸡蛋和面包放在一起，这样当顾客购买其中一种产品时，他们不需要走到别处去购买另一种产品。
- 向购买鸡蛋或面包的顾客做广告，以增加他们购买（配对的）另一种产品的倾向。
- 如果顾客一次性购买鸡蛋和面包，就为两者提供折扣。

**关联规则：**
    “如果购买了鸡蛋，那么购买面包的可能性是____”

也可以表示为：
- {鸡蛋} -> {面包}

<h3 align="left"><font color=#254E58>优点：</font></h3> 

- 相对快速的方法
- 在小量数据上表现良好
- 几乎不需要（如果有的话）特征工程

<h3 align="left"><font color=#254E58>衡量关联性的三种方式：</font></h3> 

1. 支持度
2. 置信度
3. 提升度

<h3 align="left"><font color=#254E58>举例说明：</font></h3>  
场景：超市共有5000笔交易

- A = 面包购买次数 = 500笔交易
- C = 鸡蛋购买次数 = 350笔交易
- (A->C) = 同时购买面包和鸡蛋的次数 = 150笔交易

<h3 align="left"><font color=#254E58>支持度：</font></h3>

- 支持度是数据集中某个 item 相对频率。它基本上表达了该 item 的受欢迎程度，由其在总销售 item 中的比例来表示。
-  item 的支持度 support 可以这样计算
``` support(A->C) = Support (A ∪ C)```

 示例：
  - 面包的支持度 = （包含面包的交易次数） / （总交易次数）
  - 面包的支持度 = 500 / 5000 = 0.1

<h3 align="left"><font color=#254E58>置信度：</font></h3> 

- 置信度是在数据中给定也包含前件（"if" 项）的情况下，看到后件（"then" 项）的概率
- 换句话说，置信度告诉你给定另一个 item 已购买的情况下，一个 item 被购买的可能性是多少
- 置信度决定了数据集中有多少 if-then 语句被证实为真

``` Confidence(A -> C) = (Support(A -> C)) / (Support(A))```

    其中
    A - 前件
    C - 后件

- 使用同样的例子，如果购买了面包，购买鸡蛋的可能性是：
- Confidence(面包 -> 鸡蛋) = (150/5000) / (500/5000) = 0.3 = 30%
- 因此，如果购买了面包，有 30% 的可能性会购买鸡蛋。

<h3 align="left"><font color=#254E58>提升度：</font></h3>

- 提升度是一个衡量前件和后件一起出现频率与它们独立出现频率的比率的指标。

-  ``` Lift(A -> C) = (Confidence(A -> C)) / (Support(C))```

- 提升度 > 1: A 与 C 高度相关。如果购买了 A，那么很可能也会购买 C。
- 提升度 < 1: 如果购买了 A，那么购买 C 的可能性不大。
- 提升度 = 1: 表示 item A 和 C 之间没有关联。
- 提升度(面包 -> 鸡蛋) = 0.3 / (350 / 5000) = 4.28
- 以 4.28 的提升度，你的规则将是“如果顾客购买面包，那么他们很可能会购买鸡蛋”。

Apriori算法是用来在结构化数据上实施关联规则挖掘的算法。


参考：

- [Frequent Pattern Mining](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html)
- [Market Basket Analysis with Apriori Algorithm](https://www.kaggle.com/code/prasad22/market-basket-analysis-with-apriori-algorithm)